In [1]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

from paths import *
from scpts import manipulation
from scpts import io
from scpts import caged

from basedosdados import Storage, Table, Dataset
import basedosdados as bd
import py7zr




### Download Dados
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED/

In [ ]:
raw_path = '../data/caged/raw/'
clean_path = '../data/caged/clean/'

In [23]:
anos = [i for i in range(2007,2020)]
meses= ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

In [7]:
for ano in anos:
    for mes in meses:
        download_caged_file(ano,mes,raw_path)

01/2007 ja existe
02/2007 ja existe
03/2007 ja existe
04/2007 ja existe
05/2007 ja existe
06/2007 ja existe
07/2007 ja existe
08/2007 ja existe
09/2007 ja existe
10/2007 ja existe
11/2007 ja existe
12/2007 ja existe
01/2008 ja existe
02/2008 ja existe
03/2008 ja existe
04/2008 ja existe
05/2008 ja existe
06/2008 ja existe
07/2008 ja existe
08/2008 ja existe
09/2008 ja existe
10/2008 ja existe
11/2008 ja existe
12/2008 ja existe
01/2009 ja existe
02/2009 ja existe
03/2009 ja existe
04/2009 ja existe
05/2009 ja existe
06/2009 ja existe
07/2009 ja existe
08/2009 ja existe
09/2009 ja existe
10/2009 ja existe
11/2009 ja existe
12/2009 ja existe
01/2010 ja existe
02/2010 ja existe
03/2010 ja existe
04/2010 ja existe
05/2010 ja existe
06/2010 ja existe
07/2010 ja existe
08/2010 ja existe
09/2010 ja existe
10/2010 ja existe
11/2010 ja existe
12/2010 ja existe
01/2011 ja existe
02/2011 ja existe
03/2011 ja existe
04/2011 ja existe
05/2011 ja existe
06/2011 ja existe
07/2011 ja existe
08/2011 ja

### Padronizacao

- o arquivo CAGED_062012 teve troca de nome quando foi descompactado
&nbsp;
- Particao | ano=XXXX/mex=XX/sigla_uf=XX

In [28]:
###Cria Samples de 10 rows de cada arquivo
raw_path = '../data/caged/raw/'

anos = [ano for ano in os.listdir(raw_path) if len(ano)==4]

for ano in anos:
    options = os.listdir(raw_path + f"{ano}")
    for option in options:
        meses = os.listdir(raw_path + f"{ano}/" + f"{option}/")
        for mes in meses:

            path_month = raw_path + f"{ano}/"+ f"{option}/" + f"{int(mes)}/"
            filename = [file for file in os.listdir(path_month) if '.7z' in file][0][:-3]

            if os.path.exists(f'{path_month}{filename}.csv'):
                pass
            else:
                archive = py7zr.SevenZipFile(f'{path_month}{filename}.7z', mode='r') \
                           .extractall(path=path_month)

                filename_txt = [file for file in os.listdir(path_month) if '.txt' in file][0][:-4]

                df = pd.read_csv(f'{path_month}{filename_txt}.txt',
                     sep=';', 
                     encoding='latin-1',
                     nrows=10)
                df.columns = manipulation.normalize_cols(df.columns)

                df.to_csv(f'{path_month}{filename}.csv', index=False, encoding='utf-8')

                os.remove(f'{path_month}{filename_txt}.txt')

In [62]:
raw_path = '../data/caged/raw/'

# anos = [ano for ano in os.listdir(raw_path) if len(ano)==4]
anos = [i for i in range(2007,2020)]
all_cols = pd.DataFrame()

for ano in anos:
    options = os.listdir(raw_path + f"{ano}")
    for option in options:
        meses = os.listdir(raw_path + f"{ano}/" + f"{option}/")
        for mes in meses:
            path_month = raw_path + f"{ano}/"+ f"{option}/" + f"{int(mes)}/"
            filename = [file for file in os.listdir(path_month) if '.csv' in file][0][:-4]
            df = pd.read_csv(f'{path_month}{filename}.csv')
            
            
            ## cria colunas que nao existem em outros arquivos
            check_cols = ['ind_trab_parcial', 'ind_trab_intermitente']
            create_cols = [col for col in check_cols if col not in df.columns.tolist()]

            for col in create_cols:
                df[col]=np.nan
                
            ###inicia padronizacao dos dados
            
            
            
            
            
            
            
            
            #checa nome de todas as colunas
            dd = pd.DataFrame(df.columns.tolist(),columns=['cols'])
            dd = dd.transpose().reset_index(drop=True)
            cols = dd.columns.tolist()
            dd['ano']=ano
            dd['mes']=mes
            dd['option']=option

            dd = dd[['ano','mes','option'] + cols]
            
            all_cols = pd.concat([all_cols,dd])


In [65]:
df

,admitidos_desligados,competencia_declarada,municipio,ano_declarado,cbo_2002_ocupacao,cnae_10_classe,cnae_20_classe,cnae_20_subclas,faixa_empr_inicio_jan,grau_instrucao,qtd_hora_contrat,ibge_subsetor,idade,ind_aprendiz,ind_portador_defic,raca_cor,salario_mensal,saldo_mov,sexo,tempo_emprego,tipo_estab,tipo_defic,tipo_mov_desagregado,uf,bairros_sp,bairros_fortaleza,bairros_rj,distritos_sp,regioes_adm_df,mesorregiao,microrregiao,regiao_adm_rj,regiao_adm_sp,regiao_corede,regiao_corede_04,regiao_gov_sp,regiao_senac_pr,regiao_senai_pr,regiao_senai_sp,subregiao_senai_pr,ind_trab_parcial,ind_trab_intermitente
0,1,201909,110007,2019,621005,1155,1156,115600,1,7,44,25,36,0,0,8,"001300,00",1,1,"0000,0",3,0,1,11,{ñ c,{ñ c,{ñ c,{ñ c,0,1102,11008,{ñ,999,{ñ,99,999,99,{ñ,999,99,0,0
1,1,201909,110080,2019,514320,1155,1156,115600,1,7,44,25,55,0,0,8,"000998,00",1,1,"0000,0",3,0,1,11,{ñ c,{ñ c,{ñ c,{ñ c,0,1101,11001,{ñ,999,{ñ,99,999,99,{ñ,999,99,0,0
2,1,201909,110080,2019,514320,1155,1156,115600,1,7,44,25,27,0,0,8,"001497,00",1,1,"0000,0",3,0,1,11,{ñ c,{ñ c,{ñ c,{ñ c,0,1101,11001,{ñ,999,{ñ,99,999,99,{ñ,999,99,0,0
3,1,201909,110030,2019,641015,1155,1156,115600,1,7,44,25,38,0,0,2,"001500,00",1,1,"0000,0",3,0,1,11,{ñ c,{ñ c,{ñ c,{ñ c,0,1102,11007,{ñ,999,{ñ,99,999,99,{ñ,999,99,0,0
4,1,201909,110004,2019,622005,1619,1610,161099,1,4,44,25,50,0,0,9,"001375,00",1,1,"0000,0",3,0,1,11,{ñ c,{ñ c,{ñ c,{ñ c,0,1102,11006,{ñ,999,{ñ,99,999,99,{ñ,999,99,0,0
5,1,201909,110140,2019,623110,1112,1113,111399,1,6,44,25,22,0,0,6,"001500,00",1,1,"0000,0",3,0,1,11,{ñ c,{ñ c,{ñ c,{ñ c,0,1102,11003,{ñ,999,{ñ,99,999,99,{ñ,999,99,0,0
6,1,201909,110030,2019,621005,1155,1156,115600,2,4,44,25,23,0,0,2,"001500,00",1,1,"0000,0",1,0,1,11,{ñ c,{ñ c,{ñ c,{ñ c,0,1102,11007,{ñ,999,{ñ,99,999,99,{ñ,999,99,0,0
7,1,201909,110020,2019,521115,1112,1113,111301,2,5,35,25,36,0,0,9,"001128,00",1,2,"0000,0",1,0,1,11,{ñ c,{ñ c,{ñ c,{ñ c,0,1101,11001,{ñ,999,{ñ,99,999,99,{ñ,999,99,0,0
8,1,201909,110030,2019,514320,1155,1156,115600,2,4,44,25,21,0,0,2,"001000,00",1,2,"0000,0",1,0,1,11,{ñ c,{ñ c,{ñ c,{ñ c,0,1102,11007,{ñ,999,{ñ,99,999,99,{ñ,999,99,0,0
9,1,201909,110030,2019,622420,2119,2101,210106,2,5,44,25,22,0,0,2,"000998,00",1,2,"0000,0",1,0,1,11,{ñ c,{ñ c,{ñ c,{ñ c,0,1102,11007,{ñ,999,{ñ,99,999,99,{ñ,999,99,0,0
